In [85]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
import json
import random
import scipy
import sklearn
import string
from sklearn import linear_model

In [1]:
pip install ucimlrepo

  Obtaining dependency information for ucimlrepo from https://files.pythonhosted.org/packages/3b/07/1252560194df2b4fad1cb3c46081b948331c63eb1bb0b97620d508d12a53/ucimlrepo-0.0.7-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [65]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
connect_4 = fetch_ucirepo(id=26) 
  
# data (as pandas dataframes) 
X = connect_4.data.features 
y = connect_4.data.targets 
  
# metadata 
print(connect_4.metadata) 
  
# variable information 
print(connect_4.variables) 

{'uci_id': 26, 'name': 'Connect-4', 'repository_url': 'https://archive.ics.uci.edu/dataset/26/connect+4', 'data_url': 'https://archive.ics.uci.edu/static/public/26/data.csv', 'abstract': 'Contains connect-4 positions', 'area': 'Games', 'tasks': ['Classification'], 'characteristics': ['Multivariate', 'Spatial'], 'num_instances': 67557, 'num_features': 42, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1995, 'last_updated': 'Sat Mar 09 2024', 'dataset_doi': '10.24432/C59P43', 'creators': ['John Tromp'], 'intro_paper': None, 'additional_info': {'summary': 'This database contains all legal 8-ply positions in the game of connect-4 in which neither player has won yet, and in which the next move is not forced.\r\n\r\nx is the first player; o the second.\r\n\r\nThe outcome class is the game theoretical value for the first player.', 'purpose': None, 'funded_b

In [66]:
X

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,...,f3,f4,f5,f6,g1,g2,g3,g4,g5,g6
0,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,b
1,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,b
2,b,b,b,b,b,b,o,b,b,b,...,b,b,b,b,b,b,b,b,b,b
3,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,b
4,o,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67552,x,x,b,b,b,b,o,x,o,b,...,b,b,b,b,o,o,x,b,b,b
67553,x,x,b,b,b,b,o,b,b,b,...,b,b,b,b,o,x,o,o,x,b
67554,x,x,b,b,b,b,o,o,b,b,...,b,b,b,b,o,x,x,o,b,b
67555,x,o,b,b,b,b,o,b,b,b,...,b,b,b,b,o,x,o,x,x,b


In [67]:
connect4_data = X
connect4_data['outcome'] = y['class']

In [68]:
connect4_data

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,...,f4,f5,f6,g1,g2,g3,g4,g5,g6,outcome
0,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
1,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
2,b,b,b,b,b,b,o,b,b,b,...,b,b,b,b,b,b,b,b,b,win
3,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
4,o,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67552,x,x,b,b,b,b,o,x,o,b,...,b,b,b,o,o,x,b,b,b,loss
67553,x,x,b,b,b,b,o,b,b,b,...,b,b,b,o,x,o,o,x,b,draw
67554,x,x,b,b,b,b,o,o,b,b,...,b,b,b,o,x,x,o,b,b,loss
67555,x,o,b,b,b,b,o,b,b,b,...,b,b,b,o,x,o,x,x,b,draw


In [69]:
connect4_dict = connect4_data.to_dict(orient='records')

In [73]:
list_features

[{'a1': 'b',
  'a2': 'b',
  'a3': 'b',
  'a4': 'b',
  'a5': 'b',
  'a6': 'b',
  'b1': 'b',
  'b2': 'b',
  'b3': 'b',
  'b4': 'b',
  'b5': 'b',
  'b6': 'b',
  'c1': 'x',
  'c2': 'o',
  'c3': 'b',
  'c4': 'b',
  'c5': 'b',
  'c6': 'b',
  'd1': 'x',
  'd2': 'o',
  'd3': 'x',
  'd4': 'o',
  'd5': 'x',
  'd6': 'o',
  'e1': 'b',
  'e2': 'b',
  'e3': 'b',
  'e4': 'b',
  'e5': 'b',
  'e6': 'b',
  'f1': 'b',
  'f2': 'b',
  'f3': 'b',
  'f4': 'b',
  'f5': 'b',
  'f6': 'b',
  'g1': 'b',
  'g2': 'b',
  'g3': 'b',
  'g4': 'b',
  'g5': 'b',
  'g6': 'b',
  'outcome': 'win'},
 {'a1': 'b',
  'a2': 'b',
  'a3': 'b',
  'a4': 'b',
  'a5': 'b',
  'a6': 'b',
  'b1': 'b',
  'b2': 'b',
  'b3': 'b',
  'b4': 'b',
  'b5': 'b',
  'b6': 'b',
  'c1': 'x',
  'c2': 'b',
  'c3': 'b',
  'c4': 'b',
  'c5': 'b',
  'c6': 'b',
  'd1': 'x',
  'd2': 'o',
  'd3': 'x',
  'd4': 'o',
  'd5': 'x',
  'd6': 'o',
  'e1': 'o',
  'e2': 'b',
  'e3': 'b',
  'e4': 'b',
  'e5': 'b',
  'e6': 'b',
  'f1': 'b',
  'f2': 'b',
  'f3': 'b',
  'f

In [76]:
connect4_modified = []
for game in connect4_dict:
    modified_game = {}
    for location in game:
        if game[location] == 'b':
            modified_game[location] = 0
        if game[location] == 'x':
            modified_game[location] = 1
        if game[location] == 'o':
            modified_game[location] = 2
        if game[location] == 'win':
            modified_game[location] = 1
        if game[location] == 'loss':
            modified_game[location] = -1
        if game[location] == 'draw':
            modified_game[location] = 0
            
    connect4_modified.append(modified_game)

In [77]:
connect4_modified[0]

{'a1': 0,
 'a2': 0,
 'a3': 0,
 'a4': 0,
 'a5': 0,
 'a6': 0,
 'b1': 0,
 'b2': 0,
 'b3': 0,
 'b4': 0,
 'b5': 0,
 'b6': 0,
 'c1': 1,
 'c2': 2,
 'c3': 0,
 'c4': 0,
 'c5': 0,
 'c6': 0,
 'd1': 1,
 'd2': 2,
 'd3': 1,
 'd4': 2,
 'd5': 1,
 'd6': 2,
 'e1': 0,
 'e2': 0,
 'e3': 0,
 'e4': 0,
 'e5': 0,
 'e6': 0,
 'f1': 0,
 'f2': 0,
 'f3': 0,
 'f4': 0,
 'f5': 0,
 'f6': 0,
 'g1': 0,
 'g2': 0,
 'g3': 0,
 'g4': 0,
 'g5': 0,
 'g6': 0,
 'outcome': 1}

In [90]:
index_80 = int(len(connect4_modified) * 0.8)
X_train = [[1] + list(d.values())[:-1] for d in connect4_modified[:index_80]]
y_train = [d['outcome'] for d in connect4_modified[:index_80]]
X_test = [[1] + list(d.values())[:-1] for d in connect4_modified[index_80:]]
y_test = [d['outcome'] for d in connect4_modified[index_80:]]

In [91]:
mod = sklearn.linear_model.LogisticRegression(fit_intercept=False, class_weight='balanced')
mod.fit(X_train,y_train)

/Users/dhruvkhatri/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', fit_intercept=False)

In [92]:
predictions = mod.predict(X_test) # Binary vector of predictions
correct = predictions == y_test # Binary vector indicating which predictions were correct
sum(correct) / len(correct)

0.4462699822380107